In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.pyfunc
import os
import pandas as pd

In [3]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert X_test to a Pandas DataFrame
X_test_df = pd.DataFrame(X_test, columns=iris.feature_names)

# ✅ Convert y_test into a Pandas Series and merge it with X_test_df
X_test_df["target"] = pd.Series(y_test)

In [4]:
# Get the current working directory
script_dir = os.getcwd()
parent_dir = os.path.dirname(script_dir)

# Set MLflow tracking URI properly for Windows
mlflow_tracking_dir = os.path.join(parent_dir, "mlruns").replace("\\", "/")
mlflow.set_tracking_uri(f"file:///{mlflow_tracking_dir}")

# Ensure mlruns directory exists
if not os.path.exists(mlflow_tracking_dir):
    os.makedirs(mlflow_tracking_dir)

# Update the path to your pre-trained model
model_uri = os.path.join(mlflow_tracking_dir, "4/6ad54b3126ef41d6a0b95c0ecafcb8b2/artifacts/RandomForest_model")

print("Model URI:", model_uri)

Model URI: c:/Users/JanTilles/Documents/Turku AMK/MLOps/mlruns\4/6ad54b3126ef41d6a0b95c0ecafcb8b2/artifacts/RandomForest_model


In [5]:
# Load the MLflow model
pyfunc_model = mlflow.pyfunc.load_model(model_uri)

print("Model loaded successfully.")

# Start an MLflow run before calling evaluate
with mlflow.start_run():
    # ✅ Pass the name of the target column instead of NumPy array
    validation_results = mlflow.models.evaluate(
        model=pyfunc_model,
        data=X_test_df,
        targets="target",  # ✅ Pass the column name instead of NumPy array
        model_type="classifier",
    )

    print("Validation Results:", validation_results)

# Making predictions on test data
predictions = pyfunc_model.predict(X_test_df.drop(columns=["target"]))  # Drop target before prediction
print("Predictions on test data:", predictions)


Model loaded successfully.


c:\Users\JanTilles\anaconda3\envs\ml-env\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/02/14 12:22:11 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set t

Validation Results: <mlflow.models.evaluation.base.EvaluationResult object at 0x000002246C827650>
Predictions on test data: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
